In [23]:
import psycopg2 as psy
import requests
import re
from bs4 import BeautifulSoup
from config import configr
from urllib.parse import quote_plus

In [9]:
def connect():
    connection = None
    try: 
        params = configr()
        print('Connection to the PostgreSQL databse...')
        connection = psy.connect(**params)

        # create a cursor
        cursor = connection.cursor()
        print('PostfreSQL database version: ')
        cursor.execute("Select version()")
        db_version = cursor.fetchone()
        print(db_version)
        cursor.close()
    except(Exception, psy.DatabaseError) as err:
        print(err)
    finally:
        if connection is not None:
            connection.close()
            print('Database connection terminated.')

if __name__ == "__main__":
    connect()

Connection to the PostgreSQL databse...
PostfreSQL database version: 
('PostgreSQL 16.4, compiled by Visual C++ build 1940, 64-bit',)
Database connection terminated.


In [ ]:
# Fetch Books data and insert records into Books Database

from urllib.parse import quote_plus

params = configr()
conn = psy.connect(**params)
cursor = conn.cursor()

# Extract Date from Date String
def extract_year(date_str):
    match = re.search(r'\b(19|20)\d{2}\b', date_str)
    if match:
        return int(match.group(0))
    return None

def fetch_books_by_subject(subject, batch_size=100, max_books=100):
    books_data = []
    page = 0

    while len(books_data) < max_books:
        # Fetch books from Open Library API
        url = f"https://openlibrary.org/subjects/{subject}.json?limit={batch_size}&offset={page * batch_size}"
        print(f"Requesting data from URL: {url}")

        response = requests.get(url)
        print(f"Response status code: {response.status_code}")

        if response.status_code != 200:
            print("Error fetching data from the API")
            break

        data = response.json()

        if 'works' not in data:
            print("No 'works' key in the response data. Check the subject or API documentation.")
            break

        for book_info in data.get('works', []):
            title = book_info.get("title", "")[:255]  # Truncate to 255 characters
            author = ', '.join([author['name'] for author in book_info.get("authors", [])])[:255]  # Truncate to 255 characters
            genre = subject[:100]  # Assuming genre column has enough length
            year_published_str = book_info.get("first_publish_year", "")
            year_published = extract_year(str(year_published_str)) if year_published_str else None

            # Fetch description from Google Books API
            description = fetch_book_description_from_google_books(title)

            book = {
                "title": title,
                "author": author,
                "genre": genre,
                "year_published": year_published,
                "summary": description
            }
            books_data.append(book)

            if len(books_data) >= max_books:
                break

        if len(data.get('works', [])) < batch_size:
            print("No more books to fetch from the API.")
            break

        page += 1

    print(f"Fetched {len(books_data)} books for subject: {subject}")
    return books_data

def fetch_book_description_from_google_books(title):
    query = quote_plus(title)  # Encode the title to handle spaces and special characters
    url = f"https://www.googleapis.com/books/v1/volumes?q={query}&key=<yourKey>"
    print(f"Requesting data from URL: {url}")

    response = requests.get(url)
    if response.status_code != 200:
        print("Error fetching data from Google Books API")
        return "No description available."

    data = response.json()
    items = data.get('items', [])

    if items:
        book_info = items[0].get('volumeInfo', {})
        return book_info.get('description', 'No description available.')
    
    return "No description available."

def insert_books(books):
    if not books:
        print("No books to insert.")
        return

    # Delete all existing records from the table
    cursor.execute("DELETE FROM books")
    conn.commit()
    print("Deleted all existing records from the table.")

    book_records = [
        (
            book['title'], 
            book['author'], 
            book['genre'], 
            book['year_published'], 
            book['summary']
        ) for book in books
    ]
    
    cursor.executemany("""
        INSERT INTO books (title, author, genre, year_published, summary)
        VALUES (%s, %s, %s, %s, %s)
    """, book_records)
    
    conn.commit()
    print(f"Inserted {len(book_records)} books into the database.")

def main():
    genres = [
        "fiction", "mystery", "science", "history", "biography", "fantasy", 
        "romance", "non-fiction", "horror", "adventure",
        "science-fiction", "thriller", "poetry", "philosophy", "travel", 
        "humor", "art", "religion", "spirituality"
    ]

    all_books = []
    for genre in genres:
        print(f"Fetching books for genre: {genre}")
        books = fetch_books_by_subject(genre, batch_size=100, max_books=200)
        all_books.extend(books)
        if len(all_books) >= 2000:
            break

    # Limit to a total of 1000 records
    all_books = all_books[:1000]
    
    print(f"Total books fetched: {len(all_books)}")
    insert_books(all_books)

    # Verify insertion
    cursor.execute("SELECT COUNT(*) FROM books")
    count = cursor.fetchone()[0]
    print(f"Total books in the database: {count}")

    # Close the connection
    cursor.close()
    conn.close()

if __name__ == "__main__":
    main()

In [ ]:
## Didn't run because of api limit reached

'''from urllib.parse import quote_plus

params = configr()
conn = psy.connect(**params)
cursor = conn.cursor()

def extract_year(date_str):
    match = re.search(r'\b(19|20)\d{2}\b', date_str)
    if match:
        return int(match.group(0))
    return None

def fetch_books_from_google_books(query, max_books=100):
    books_data = []
    page = 0

    while len(books_data) < max_books:
        # Construct query URL
        url = f"https://www.googleapis.com/books/v1/volumes?q={quote_plus(query)}&startIndex={page * 10}&maxResults=10&key=<your key>"
        print(f"Requesting data from URL: {url}")

        response = requests.get(url)
        print(f"Response status code: {response.status_code}")

        if response.status_code != 200:
            print("Error fetching data from the API")
            break

        data = response.json()

        items = data.get('items', [])
        if not items:
            print("No more books to fetch from the API.")
            break

        for item in items:
            volume_info = item.get('volumeInfo', {})
            title = volume_info.get("title", "")[:255]  # Truncate to 255 characters
            author = ', '.join(volume_info.get("authors", []))[:255]  # Truncate to 255 characters
            genre = ', '.join(volume_info.get("categories", []))[:100]  # Assuming genre column has enough length
            year_published_str = volume_info.get("publishedDate", "")
            year_published = extract_year(year_published_str) if year_published_str else None
            description = volume_info.get('description', None)  # Use None if no description is available

            # Only add books with a description
            if description:
                book = {
                    "title": title,
                    "author": author,
                    "genre": genre,
                    "year_published": year_published,
                    "summary": description[:1000]  # Truncate to 1000 characters
                }
                books_data.append(book)

            if len(books_data) >= max_books:
                break

        page += 1

    print(f"Fetched {len(books_data)} books for query: {query}")
    return books_data

def insert_books(books):
    if not books:
        print("No books to insert.")
        return

    book_records = [
        (
            book['title'], 
            book['author'], 
            book['genre'], 
            book['year_published'], 
            book['summary']
        ) for book in books
    ]
    
    cursor.executemany("""
        INSERT INTO books (title, author, genre, year_published, summary)
        VALUES (%s, %s, %s, %s, %s)
    """, book_records)
    
    conn.commit()
    print(f"Inserted {len(book_records)} books into the database.")

def main():
    queries = [
        "fiction", "mystery", "science", "history", "biography", "fantasy", 
        "romance", "non-fiction", "horror", "adventure",
        "science-fiction", "thriller", "poetry", "philosophy", "travel", 
        "humor","art", "religion", "spirituality"
    ]

    all_books = []
    for query in queries:
        print(f"Fetching books for query: {query}")
        books = fetch_books_from_google_books(query, max_books=100)
        all_books.extend(books)
        if len(all_books) >= 1000:
            break

        # Write records to database after processing each genre
        insert_books(books)

    # Limit to a total of 1000 records
    all_books = all_books[:1000]
    
    print(f"Total books fetched: {len(all_books)}")

    # Write remaining records to the database
    if all_books:
        insert_books(all_books)

    # Verify insertion
    cursor.execute("SELECT COUNT(*) FROM books")
    count = cursor.fetchone()[0]
    print(f"Total books in the database: {count}")

    # Close the connection
    cursor.close()
    conn.close()

if __name__ == "__main__":
    main()
'''

In [45]:
# Fetch reviews and insert records into Review Table

params = configr()
conn = psy.connect(**params)
cursor = conn.cursor()

cursor.execute("SET client_encoding = 'UTF8';")
conn.commit()

# Function to clean and encode text
def clean_text(text):
    if isinstance(text, str):
        try:
            return text.encode('utf-8', 'ignore').decode('utf-8')
        except UnicodeDecodeError:
            return text
    return text

# Function to fetch book IDs and titles from the `books` table
def fetch_book_ids_and_titles():
    cursor.execute("SELECT id, title FROM books")
    return cursor.fetchall()

# Function to fetch reviews and rating from Goodreads for a specific book
def fetch_reviews_and_rating(book_id, book_title, max_retries=3):
    reviews_data = []
    
    # Use direct book page URL
    url = f"https://www.goodreads.com/book/show/{book_id}"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }

    retries = 0
    while retries < max_retries:
        try:
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()
            
            soup = BeautifulSoup(response.text, 'html.parser')
            
            # Extract ratings
            rating_elements = soup.find_all('span', class_='RatingStars')
            ratings = [elem.get('aria-label', '').split(' ')[1] for elem in rating_elements]
            
            # Find all review sections
            review_elements = soup.find_all('span', class_='Formatted')
            if review_elements:
                print(f"Found {len(review_elements)} reviews")
            else:
                print("No reviews found")

            for i, review in enumerate(review_elements, 1):
                review_text = clean_text(review.get_text(strip=True))
                
                # Use the corresponding rating for each review
                rating = ratings[i-1] if i-1 < len(ratings) else None
                
                # Validate rating value
                if rating is not None:
                    try:
                        rating = float(rating)
                        if rating < 1 or rating > 5:  
                            rating = None
                    except ValueError:
                        rating = None
                else:
                    rating = None
                
                if review_text and rating is not None:
                    reviews_data.append({
                        "book_id": book_id,
                        "user_id": 1,
                        "review_text": review_text,
                        "rating": rating
                    })
                    if len(reviews_data) % 50 == 0:
                        print(f"Review {len(reviews_data)}: {review_text}\nRating: {rating}\n")
                    
            break  # Exit loop if successful
        except requests.exceptions.RequestException as e:
            print(f"Error fetching reviews for book ID '{book_id}': {e}")
            retries += 1
            if retries < max_retries:
                print(f"Retrying... ({retries}/{max_retries})")
                time.sleep(2)  # Wait 2 seconds before retrying
            else:
                print(f"Failed to fetch reviews for book ID '{book_id}' after {max_retries} attempts.")
    
    return reviews_data


# Function to insert reviews into the `review` table
def insert_reviews(reviews):
    try:
        if not reviews:
            print("No reviews to insert.")
            return

        batch_size = 100
        for i in range(0, len(reviews), batch_size):
            batch = reviews[i:i + batch_size]
            for review in batch:
                # Convert rating to float
                try:
                    rating = float(review['rating'])
                except ValueError:
                    print(f"Invalid rating value: {review['rating']}")
                    continue

                cursor.execute("""
                    INSERT INTO review (book_id, user_id, review_text, rating)
                    VALUES (%s, %s, %s, %s)
                    """, (review['book_id'], review['user_id'], review['review_text'], rating)
                )
            conn.commit()
            print(f"Inserted {len(batch)} reviews into the database.")
    except psy.DatabaseError as e:
        print(f"Database error: {e}")
        conn.rollback()  # Rollback in case of an error

# Main function to process all books
def main():
    books = fetch_book_ids_and_titles()
    if not books:
        print("No books found in the database.")
        return

    all_reviews = []
    for index, (book_id, book_title) in enumerate(books, 1):
        print(f"Fetching reviews for book ID {book_id}: {book_title}")
        reviews = fetch_reviews_and_rating(book_id, book_title)
        all_reviews.extend(reviews)

        # Print status for every 50th book
        if index % 50 == 0:
            print(f"Processed {index} books.")

        # Insert reviews into the database every 100 records
        if len(all_reviews) >= 100:
            insert_reviews(all_reviews)
            all_reviews = []  # Reset reviews list after insertion

    # Insert any remaining reviews
    if all_reviews:
        insert_reviews(all_reviews)

    print("Processing completed for all books.")

# Execute the main function
if __name__ == "__main__":
    main()

# Close the connection
cursor.close()
conn.close()


Fetching reviews for book ID 24277: Under the Greenwood Tree or, The Mellstock quire
Found 7 reviews
Fetching reviews for book ID 24083: Pride and Prejudice
Found 33 reviews
Fetching reviews for book ID 24278: Kidnapped
Found 3 reviews
Fetching reviews for book ID 24365: Cat Among the Pigeons
Found 33 reviews
Fetching reviews for book ID 25011: Pictures from Italy
Found 4 reviews
Fetching reviews for book ID 24084: Alice's Adventures in Wonderland
No reviews found
Fetching reviews for book ID 24085: Adventures of Huckleberry Finn
Found 3 reviews
Fetching reviews for book ID 24086: Emma
Found 6 reviews
Fetching reviews for book ID 24087: Frankenstein or The Modern Prometheus
Found 33 reviews
Fetching reviews for book ID 24088: The Picture of Dorian Gray
Found 33 reviews
Inserted 100 reviews into the database.
Inserted 15 reviews into the database.
Fetching reviews for book ID 24089: Wuthering Heights
Found 33 reviews
Fetching reviews for book ID 24090: Sense and Sensibility
Found 3 revi

KeyboardInterrupt: 

In [38]:
# params = configr()
# conn = psy.connect(**params)
# cursor = conn.cursor()

# cursor.execute("SET client_encoding = 'UTF8';")
# conn.commit()

# # Function to clean and encode text
# def clean_text(text):
#     if isinstance(text, str):
#         try:
#             return text.encode('utf-8', 'ignore').decode('utf-8')
#         except UnicodeDecodeError:
#             return text
#     return text

# # Function to fetch book IDs and titles from the `books` table
# def fetch_book_ids_and_titles(limit=1):
#     cursor.execute("SELECT id, title FROM books LIMIT %s", (limit,))
#     return cursor.fetchall()

# # Function to fetch reviews and rating from Goodreads for a specific book
# def fetch_reviews_and_rating(book_id, book_title, max_retries=3):
#     reviews_data = []
    
#     # Use direct book page URL
#     url = f"https://www.goodreads.com/book/show/{book_id}"
#     headers = {
#         'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
#     }

#     retries = 0
#     while retries < max_retries:
#         try:
#             response = requests.get(url, headers=headers, timeout=10)
#             response.raise_for_status()
            
#             soup = BeautifulSoup(response.text, 'html.parser')
            
#             # Print raw HTML for debugging
#             # Uncomment this line to debug the HTML structure
#             # print(soup.prettify())

#             # Find the rating
#             rating_element = soup.find('div', class_='RatingStatistics__column')
#             if rating_element:
#                 aria_label = rating_element.get('aria-label', '')
#                 if aria_label:
#                     rating = aria_label.split(' ')[-2]  # Extract rating value
#                     print(f"Rating: {rating}")
#                 else:
#                     rating = 'Rating not found'
#             else:
#                 rating = 'Rating section not found'
#             print(f"Rating: {rating}")

#             # Find all review sections
#             review_elements = soup.find_all('span', class_='Formatted')
#             if review_elements:
#                 print(f"Found {len(review_elements)} reviews")
#             else:
#                 print("No reviews found")

#             for i, review in enumerate(review_elements, 1):
#                 review_text = clean_text(review.get_text(strip=True))
                
#                 if review_text:
#                     reviews_data.append({
#                         "book_id": book_id,
#                         "user_id": 1,  # Assuming a fixed user ID for this example
#                         "review_text": review_text,
#                         "rating": rating
#                     })
#                     print(f"Review {i}: {review_text}\n")
                    
#             print(f"Fetched reviews for '{book_title}': {reviews_data}")  # Print fetched reviews
#             break  # Exit loop if successful
#         except requests.exceptions.RequestException as e:
#             print(f"Error fetching reviews for book ID '{book_id}': {e}")
#             retries += 1
#             if retries < max_retries:
#                 print(f"Retrying... ({retries}/{max_retries})")
#                 time.sleep(2)  # Wait 2 seconds before retrying
#             else:
#                 print(f"Failed to fetch reviews for book ID '{book_id}' after {max_retries} attempts.")
    
#     return reviews_data

# # Function to insert reviews into the `reviews` table
# # Function to insert reviews into the `reviews` table
# def insert_reviews(reviews):
#     try:
#         if not reviews:
#             print("No reviews to insert.")
#             return

#         for review in reviews:
#             # Convert rating to float
#             try:
#                 rating = float(review['rating'])
#             except ValueError:
#                 print(f"Invalid rating value: {review['rating']}")
#                 continue

#             cursor.execute("""
#                 INSERT INTO reviews (book_id, user_id, review_text, rating)
#                 VALUES (%s, %s, %s, %s)
#                 """, (review['book_id'], review['user_id'], review['review_text'], rating)
#             )
#         conn.commit()
#         print(f"Inserted {len(reviews)} reviews into the database.")
#     except psy.DatabaseError as e:
#         print(f"Database error: {e}")
#         conn.rollback()  # Rollback in case of an error


# # Main function to process the first book only
# def main():
#     # Fetch only the first book
#     books = fetch_book_ids_and_titles(limit=1)
#     if not books:
#         print("No books found in the database.")
#         return

#     book_id, book_title = books[0]
#     print(f"Fetching reviews for book ID {book_id}: {book_title}")
#     reviews = fetch_reviews_and_rating(book_id, book_title)
#     insert_reviews(reviews)

#     print("Processing completed for the first book.")

# # Execute the main function
# if __name__ == "__main__":
#     main()

# # Close the connection
# cursor.close()
# conn.close()


Fetching reviews for book ID 24277: Under the Greenwood Tree or, The Mellstock quire
Rating: 4.18
Rating: 4.18
Found 7 reviews
Review 1: NEW EDITION 'Magic that takes you out, far out, of this time and this world.' George Bernard Shaw, after a visit to Skellig This is the story of two of the world's most stunning and unspoilt islands, Skellig Michael and Small Skellig, which lie off the coast of Kerry. Lavelle explores the extraordinary, isolated Early Christian monastic settlement with its stone 'beehive' huts. He describes the abundant bird life, including the huge colony of gannets, and tells of the history, the legends, geology, plant life, the lighthouse, the seals and the underwater world. A comprehensive and accessible book on a unique and fascinating place.

Review 4: This fairly short book provides a nice overview of the Skelligs by an author who clearly has a deep interest and knowledge of all aspects of these unique islands. The topography, geology and flora and fauna of the